In [293]:
import os

PROJECT_ID = "swift-kiln-380818"
LOCATION = "us-central1"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/aditygrg/.config/gcloud/application_default_credentials.json"

import vertexai
from vertexai.generative_models import GenerativeModel, ChatSession

vertexai.init(location=LOCATION, project=PROJECT_ID)

In [294]:
import requests
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    SafetySetting,
    HarmCategory,
    HarmBlockThreshold,
    Part,
    Tool,
)
from vertexai.preview import rag

In [295]:
user_verified = FunctionDeclaration(
    name="user_verified",
    description="This is called when the user is successfully verified",
    parameters={
        "type": "object",
        "properties": {"status": {"type": "boolean"}},
    },
)

get_user_data = FunctionDeclaration(
    name="get_user_data",
    description="Get the info of the user to verify",
    parameters={
        "type": "object",
        "properties": {"phone_number": {"type": "string"}},
    },
)

user_not_verified = FunctionDeclaration(
    name="user_not_verified",
    description="This is called when user is not verified",
    parameters={
        "type": "object",
        "properties": {"status": {"type": "boolean"}},
    },
)

In [296]:
#Implementation 1

retail_tool = Tool(
    function_declarations=[
        get_user_data,
        user_verified,
        user_not_verified
    ],
)

In [297]:
SYSTEM_INSTRUCTION = """
You are an Amazon Customer Support Agent and you are compassionate and assuring to help your user always!

You need to always verify if you are talking to the same person as the account holder for security purposes.

If there is no user_data, you can ask for his phone number and input it to the `find_data` function to get the data.

After you have the data, you can verify then by asking pincode, city and state of the customer and matching it with the data.


Only move ahead with query once you are able to verify pincode and address of the customer.

Always start with: "Welcome to Amazon! and a good greeting"
"""

In [298]:
model = GenerativeModel(
    "gemini-1.5-pro-001",
    generation_config=GenerationConfig(temperature=0.3),
    tools=[retail_tool],
    system_instruction=SYSTEM_INSTRUCTION
)
chat = model.start_chat(response_validation=False)

In [299]:
prompt = """
%User Query:

I am facing some difficulties in ordering a data.

User data:
phone_no 932489237
name "Raj Patel"
email "raja23@gmail.com"
subscriptionStatus false

previousOrders ->
order_id "ord123"
status: "Delivered"
product: 
name "HRX Oversized T-Shirt"
description "Cotton-Comfy fit Oversized T-Shirt"
category "Clothing-Men-TShirt"
average_rating 3
price 399
Pincode: 262311
City: Indore
State: Madhya Pradesh
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

text: "Welcome to Amazon! I hope you are having a good day. I understand you are facing some difficulties in ordering data. I would be happy to assist you with that. For security purposes, could you please provide me with the phone number associated with your account? \n"

In [300]:
safety_config = [
            SafetySetting(
                category=HarmCategory.HARM_CATEGORY_UNSPECIFIED,
                threshold=HarmBlockThreshold.BLOCK_NONE
            ),
            SafetySetting(
                category=HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
                threshold=HarmBlockThreshold.BLOCK_NONE
            ),
            SafetySetting(
                category=HarmCategory.HARM_CATEGORY_HARASSMENT,
                threshold=HarmBlockThreshold.BLOCK_NONE
            ),
            SafetySetting(
                category=HarmCategory.HARM_CATEGORY_HATE_SPEECH,
                threshold=HarmBlockThreshold.BLOCK_NONE
            ),
            SafetySetting(
                category=HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
                threshold=HarmBlockThreshold.BLOCK_NONE
            )
        ]


In [301]:
messages = [
    ("User", prompt),
    ("AI", "Welcome to Amazon! Hope you are having a good day.\n\nI understand you are facing some difficulties in ordering data. I am more than happy to help!\n\nBefore we proceed, for security purposes, could you please provide me with the pin code associated with your Amazon account?\n"),
    ("Human", "I dont know my pin code"),
]

In [302]:
def convert_to_str(messages):
    ans = ""
    for i in messages:
        ans += i[0] + " : " + i[1] + "\n"

    return ans

In [304]:
res = chat.send_message(convert_to_str(messages), safety_settings=safety_config)

In [305]:
res.candidates[0].content.parts[0]

function_call {
  name: "get_user_data"
  args {
    fields {
      key: "phone_number"
      value {
        string_value: "932489237"
      }
    }
  }
}

In [306]:
messages = [
    ("User", prompt),
    ("AI", "Welcome to Amazon! Hope you are having a good day.\n\nI understand you are facing some difficulties in ordering data. I am more than happy to help!\n\nBefore we proceed, for security purposes, could you please provide me with the pin code associated with your Amazon account?\n"),
    ("Human", "I dont know my pin code"),
    ("AI", "That\'s alright! Can you please provide me with the city and state associated with your Amazon account? \n"),
    ("Human", "My city is Indore")
]

In [307]:
res = chat.send_message(convert_to_str(messages), safety_settings=safety_config)

In [308]:
res

candidates {
  content {
    role: "model"
    parts {
      text: "That\'s alright! Could you please also tell me the state associated with your Amazon account? \n"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.0987725928
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.103930376
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.318061382
    severity: HARM_SEVERITY_LOW
    severity_score: 0.341582507
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.107812636
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0872522444
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.105210543
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0699218363
  }
}
usage

In [309]:
messages = [
    ("User", prompt),
    ("AI", "Welcome to Amazon! Hope you are having a good day.\n\nI understand you are facing some difficulties in ordering data. I am more than happy to help!\n\nBefore we proceed, for security purposes, could you please provide me with the pin code associated with your Amazon account?\n"),
    ("Human", "I dont know my pin code"),
    ("AI", "That\'s alright! Can you please provide me with the city and state associated with your Amazon account? \n"),
    ("Human", "My city is Indore"),
    ("AI", "That\'s alright! Could you please also tell me the state associated with your Amazon account? \n"),
    ("Human", "Madhya Pradesh")
]

In [310]:
res = chat.send_message(convert_to_str(messages), safety_settings=safety_config)

In [311]:
res

candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "get_user_data"
        args {
          fields {
            key: "phone_number"
            value {
              string_value: "932489237"
            }
          }
        }
      }
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.192084283
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.114963353
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: MEDIUM
    probability_score: 0.751648128
    severity: HARM_SEVERITY_MEDIUM
    severity_score: 0.544619739
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.169131249
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.134094119
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probab

In [ ]:
response_confirm = res

In [ ]:
print(not response_confirm.candidates[0].content.parts[0].function_call)

True


In [312]:
res.candidates[0]

content {
  role: "model"
  parts {
    function_call {
      name: "get_user_data"
      args {
        fields {
          key: "phone_number"
          value {
            string_value: "932489237"
          }
        }
      }
    }
  }
}
finish_reason: STOP
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
  probability_score: 0.192084283
  severity: HARM_SEVERITY_NEGLIGIBLE
  severity_score: 0.114963353
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: MEDIUM
  probability_score: 0.751648128
  severity: HARM_SEVERITY_MEDIUM
  severity_score: 0.544619739
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
  probability_score: 0.169131249
  severity: HARM_SEVERITY_NEGLIGIBLE
  severity_score: 0.134094119
}
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
  probability_score: 0.207857162
  severity: HARM_SEVERITY_NEGLIGIBLE
  severity_score: 0.0726367
}

In [ ]:
response = chat.send_message(
    Part.from_function_response(
        name="user_not_verified",
        response={
            "content": "Sorry, we were unable to verify your details. Thank you for contacting Amazon!",
        },
    ),
)
response.candidates[0].content.parts[0]

text: "Sorry, we were unable to verify your details. Thank you for contacting Amazon! \n"

In [ ]:
prompt = """
What about the Pixel 8? Is there a store in
Mountain View, CA that I can visit to try one out?
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

In [ ]:
# Here you can use your preferred method to make an API request and get a response.
# In this example, we'll use synthetic data to simulate a payload from an external API response.

api_response = {"store": "2000 N Shoreline Blvd, Mountain View, CA 94043, US"}

In [ ]:
response = chat.send_message(
    Part.from_function_response(
        name="get_store_location",
        response={
            "content": api_response,
        },
    ),
)
response.candidates[0].content.parts[0]

In [ ]:
api_response = {"sku": "GA08475-US", "in_stock": "yes"}

In [ ]:
response = chat.send_message(
    Part.from_function_response(
        name="get_product_info",
        response={
            "content": api_response,
        },
    ),
)
response.candidates[0].content.parts[0]

In [ ]:
prompt = """
I'd like to order a Pixel 8 Pro and have it shipped to 1155 Borregas Ave, Sunnyvale, CA 94089.
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

In [ ]:
api_response = {
    "payment_status": "paid",
    "order_number": 12345,
    "est_arrival": "2 days",
}

In [ ]:
response = chat.send_message(
    Part.from_function_response(
        name="place_order",
        response={
            "content": api_response,
        },
    ),
)
response.candidates[0].content.parts[0]